In [1]:
pip install faker

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import random

from faker import Faker

import sqlite3

%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [6]:
N_PRODUTOS = 100
N_CLIENTES = 20
N_COMPRAS = 500

In [7]:
faker = Faker('pt_BR')
faker.name()

'Sra. Sarah Gonçalves'

In [8]:
db = sqlite3.connect('TesteSQL.db')
db

In [9]:
%sql sqlite:///TesteSQL.db

In [10]:
produtos = pd.DataFrame([[f'prod {i:02}', round(random.random() * 10, 2)] 
                         for i in range(N_PRODUTOS)], 
                        columns=['produto', 'preço'])
produtos.head()

,produto,preço
0,prod 00,4.49
1,prod 01,8.88
2,prod 02,0.68
3,prod 03,9.55
4,prod 04,7.93


In [11]:
produtos.to_sql('produtos', db, index_label='id', if_exists='replace')

100

In [12]:
pd.read_sql("SELECT * FROM produtos;", db, 'id').head()

,produto,preço
id,,
0,prod 00,4.49
1,prod 01,8.88
2,prod 02,0.68
3,prod 03,9.55
4,prod 04,7.93


In [13]:
clientes = pd.DataFrame([
    [faker.name(), faker.free_email(), random.randint(18, 60)]
for _ in range(N_CLIENTES)], columns=['nome', 'email', 'idade'])

clientes.head()

,nome,email,idade
0,Eduarda Gonçalves,lbarros@bol.com.br,39
1,Bruno da Rosa,ocardoso@yahoo.com.br,33
2,Srta. Ana Beatriz Jesus,gustavo-henrique55@yahoo.com.br,30
3,Emanuel das Neves,cardosoraquel@hotmail.com,32
4,Srta. Amanda Moreira,ana-liviacastro@ig.com.br,45


In [14]:
clientes.to_sql('clientes', db, index_label='id', if_exists='replace')
pd.read_sql("SELECT * FROM clientes;", db, 'id').head()

,nome,email,idade
id,,,
0,Eduarda Gonçalves,lbarros@bol.com.br,39
1,Bruno da Rosa,ocardoso@yahoo.com.br,33
2,Srta. Ana Beatriz Jesus,gustavo-henrique55@yahoo.com.br,30
3,Emanuel das Neves,cardosoraquel@hotmail.com,32
4,Srta. Amanda Moreira,ana-liviacastro@ig.com.br,45


In [15]:
compras = pd.DataFrame([[
    random.randint(0, N_CLIENTES - 1),
    random.randint(0, N_PRODUTOS - 1)
] for _ in range(N_COMPRAS)], columns=['id_cliente', 'id_produto'])
compras.head()

,id_cliente,id_produto
0,14,19
1,2,30
2,11,90
3,8,18
4,19,39


In [16]:
compras.to_sql('compras', db, index_label='id', if_exists='replace')
pd.read_sql("SELECT * FROM compras;", db, 'id').head()

,id_cliente,id_produto
id,,
0,14,19
1,2,30
2,11,90
3,8,18
4,19,39


In [17]:
del compras, clientes, produtos

# Select

Ao fazer uma extração num banco de dados SQL, independentemente do 
DBMS (se é Oracle, SQLite, MySQL/MariaDB e etc.), você sempre vai utilizar a 
palava `SELECT` seguida pelo que você quer trazer, e por fim a palavra FROM
informando de onde você irá coletar esses dados, por exemplo:

In [18]:
%%sql

SELECT nome, idade FROM clientes

 * sqlite:///TesteSQL.db
Done.


nome,idade
Eduarda Gonçalves,39
Bruno da Rosa,33
Srta. Ana Beatriz Jesus,30
Emanuel das Neves,32
Srta. Amanda Moreira,45
Sr. Nicolas Cardoso,39
Alícia Araújo,22
Augusto Costa,35
Marcos Vinicius Novaes,58
Brenda Porto,44


Nesse caso, eu estou trazendo o nome e idade de todos os clientes da base.

# Limit

Geralmente não queremos trazer todos os dados de uma vez, muitas vezes os dados
que guardamos nas tabelas do banco de dados iriam facilmente estourar a  memória
dos nossos servidores, para limitar a pesquisa, existem vários métodos, o mais 
simples que tem é o `LIMIT n`, onde você informa quantos registros você quer ver
no máximo.

In [19]:
%%sql

SELECT * FROM clientes LIMIT 5

 * sqlite:///TesteSQL.db
Done.


id,nome,email,idade
0,Eduarda Gonçalves,lbarros@bol.com.br,39
1,Bruno da Rosa,ocardoso@yahoo.com.br,33
2,Srta. Ana Beatriz Jesus,gustavo-henrique55@yahoo.com.br,30
3,Emanuel das Neves,cardosoraquel@hotmail.com,32
4,Srta. Amanda Moreira,ana-liviacastro@ig.com.br,45


Nesse caso, eu trouxe 5 registros da tabela cliente.

<span style="color:red"><b>Em uma aplicação, sempre use o</span></b> `LIMIT` 
<span style="color:red"><b>com um</span></b> `ORDER BY` 
<span style="color:red"><b>quer será explicado mais a frente

## Exercício 01

Traga 10 produtos e seu respectivo preço

In [20]:
%%sql

SELECT * FROM produtos LIMIT 10

 * sqlite:///TesteSQL.db
Done.


id,produto,preço
0,prod 00,4.49
1,prod 01,8.88
2,prod 02,0.68
3,prod 03,9.55
4,prod 04,7.93
5,prod 05,1.37
6,prod 06,5.04
7,prod 07,2.06
8,prod 08,1.89
9,prod 09,3.08


# Projeção

Também posso operar sobre os valores retornados, por exemplo, se ao invés da idade
da pessoa, eu quiser saber somente a faixa etária dela, posso fazer algum cálculo
no select e assim trazer esse dado já trabalhado do próprio banco de dados. 

In [21]:
%%sql

SELECT NOME, ROUND(idade / 10) * 10 AS FAIXA_ETARIA
FROM clientes

 * sqlite:///TesteSQL.db
Done.


nome,FAIXA_ETARIA
Eduarda Gonçalves,30.0
Bruno da Rosa,30.0
Srta. Ana Beatriz Jesus,30.0
Emanuel das Neves,30.0
Srta. Amanda Moreira,40.0
Sr. Nicolas Cardoso,30.0
Alícia Araújo,20.0
Augusto Costa,30.0
Marcos Vinicius Novaes,50.0
Brenda Porto,40.0


Essa ação de selecionar um dado trabalhado, recebe vários nomes, como mapeamento
ou projeção.

## Exercício 2

Traga 10 produtos e seu preço convertido para dólar, considere a cotação do dólar
com sendo US\\$1,00=R\\$5,20

In [22]:
%%sql

SELECT produto, (preço/5.20)  AS preço_convertido
FROM produtos LIMIT 10

 * sqlite:///TesteSQL.db
Done.


produto,preço_convertido
prod 00,0.8634615384615385
prod 01,1.7076923076923078
prod 02,0.13076923076923078
prod 03,1.8365384615384617
prod 04,1.525
prod 05,0.26346153846153847
prod 06,0.9692307692307692
prod 07,0.39615384615384613
prod 08,0.36346153846153845
prod 09,0.5923076923076923


# Where

O `LIMIT` como dito anteriormente é a forma mais simples de reduzir a quantidade
de dados trazidos em uma pesquisa, mas a forma mais comum de se usar é o `WHERE`.

Geralmente quando fazemos uma consulta no banco de dados, não precisamos de todos
os dados da tabela, somente dados que respeitem algum requisito, como por exemplo,
no nosso caso, se a gente quiser saber quais clientes temos com menos de 30 anos:

In [23]:
%%sql

SELECT nome, idade FROM clientes WHERE idade < 30

 * sqlite:///TesteSQL.db
Done.


nome,idade
Alícia Araújo,22
Luigi Souza,21
Isis Ramos,23


## Exercício 3

Traga todos os produtos que custem menos do que 1 real

In [24]:
%%sql

SELECT produto, preço FROM produtos WHERE preço<1

 * sqlite:///TesteSQL.db
Done.


produto,preço
prod 02,0.68
prod 16,0.6
prod 28,0.01
prod 47,0.15
prod 51,0.24
prod 59,0.88
prod 65,0.98
prod 72,0.55
prod 73,0.52
prod 80,0.69


# Join

Uma das principais características dos bancos relacionais é justamente a sua capacidade
de ter relacionamento entre os dados.

Geralmente esse relacionamento é feito utilizando chaves primárias (PK). Geralmente
as PKs são representadas por um número inteiro chamado ID, mas não necessariamente,
por exemplo, um cadastro de pessoas pode ter como PK o CPF da pessoa, o que importa
é que não haja repetição de valores.

As PKs também podem ter mais de uma coluna, por exemplo, um supermercado pode ter
como PK um produto e a marca dele.

Mas por hora iremos abstrair essas PKs menos ortodoxas e focar nas padrões que é só
uma coluna ID.

As PKs geralmente são utilizadas para formar relacionamento entre as tabelas, ou seja,
a partir do momento que eu cadastrei um cliente no meu sistema, e gerei um ID para ele
toda as interações dele serão registradas utilizando o seu ID e não mais dados
pessoais.

No nosso caso, registramos as compras utilizando o ID do cliente e do produto, como
podemos ver abaixo

In [25]:
%%sql

SELECT cl.*, po.*
FROM clientes cl
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
LIMIT 5

 * sqlite:///TesteSQL.db
Done.


id,nome,email,idade,id_1,produto,preço
14,André da Cruz,vmendes@yahoo.com.br,33,19,prod 19,5.44
2,Srta. Ana Beatriz Jesus,gustavo-henrique55@yahoo.com.br,30,30,prod 30,8.28
11,Benjamin Azevedo,catarina08@ig.com.br,43,90,prod 90,3.96
8,Marcos Vinicius Novaes,augusto45@yahoo.com.br,58,18,prod 18,9.46
19,Isis Ramos,carvalhovitor-hugo@yahoo.com.br,23,39,prod 39,1.49


A tabela de compras traz consigo o ID do cliente que fez a compra e o ID do produto
comprado, ocupando assim muito menos espaço do que se eu tivesse uma tabela com
todos os dados.

> NOTA: Quando nas colunas selecionadas faço algo do tipo alias.* eu estou trazendo todas as colunas daquela tabel

> NOTA: Em 99% dos casos, independetemente do intuito do SELECT, o relacionamento entre as tabelas vai utilizar as mesmas colunas que usou em SELECTs anteriores

## Exercício 4

Traga 10 compras do produto `prod 02`

In [26]:
%%sql

SELECT cl.*, po.*
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
WHERE produto = 'prod 02'
LIMIT 10

 * sqlite:///TesteSQL.db
Done.


id,nome,email,idade,id_1,produto,preço
4,Srta. Amanda Moreira,ana-liviacastro@ig.com.br,45,2,prod 02,0.68
11,Benjamin Azevedo,catarina08@ig.com.br,43,2,prod 02,0.68


# Distinct

Muitas vezes as nossas consultas nos trazem resultados repetidos, quando a repetição
não nos importa, por exemplo, mesmo na nossa base tendo 20 clientes, não necessariamente
todos efetuaram compras, se eu quiser saber todos que o fizeram posso fazer:

In [27]:
%%sql

SELECT DISTINCT cl.nome
FROM clientes cl
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID

 * sqlite:///TesteSQL.db
Done.


nome
André da Cruz
Srta. Ana Beatriz Jesus
Benjamin Azevedo
Marcos Vinicius Novaes
Isis Ramos
Rafaela Fernandes
Alícia Araújo
Sr. Nicolas Cardoso
João Miguel Lopes
Sra. Laura Pinto


Como podemos ver, apesar de eu ter feito {{N_COMPRAS}} compras, o meu resultado
não traz essa quantidade de linhas, traz exatamente a quantidade de clientes 
que fizeram compras.

## Exercício 5

Liste todos clientes que compraram o produto `prod 02`

In [28]:
%%sql

SELECT DISTINCT cl.nome
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
WHERE produto = 'prod 02'


 * sqlite:///TesteSQL.db
Done.


nome
Srta. Amanda Moreira
Benjamin Azevedo


# Redução

No SQL também são comuns funções de redução, elas percorrem todas as linhas de uma
determinada coluna aplicando uma função de redução (somatório, contagem e etc.)
e retornam somente uma linha ao invés de todas as linhas lidas por ela, por exemplo
se eu quiser saber quantas compras do `prod 02` foram feitas, posso fazer:

In [29]:
%%sql

SELECT count(*)
FROM compras AS co
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
WHERE po.produto = 'prod 02'

 * sqlite:///TesteSQL.db
Done.


count(*)
2


Existem outras funções como SUM, AVG e etc., você pode sempre consultar a 
[documentação oficial](https://www.sqlite.org/lang_aggfunc.html) para ver 
todas elas

## Exercício 6

Qual foi a receita total?

In [30]:
%%sql

SELECT round(sum(po.preço)) AS receita_total
FROM compras AS co
JOIN produtos as po
ON co.ID_PRODUTO = po.ID

 * sqlite:///TesteSQL.db
Done.


receita_total
2474.0


# Group by

As funções de agregação ficam ainda mais fortes quando agrupamos valores.
Isto é, elas mostram todo o seu potencial quando criamos pequenos grupos de dados,
ao invés de contarmos por exemplo todas as vendas do dia, podemos aplicar contagem
produto a produto com somente uma query, como mostrado abaixo:

In [31]:
%%sql

SELECT po.produto, count(*)
FROM compras AS co
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by po.produto
limit 10

 * sqlite:///TesteSQL.db
Done.


produto,count(*)
prod 00,6
prod 01,2
prod 02,2
prod 03,3
prod 04,7
prod 05,8
prod 06,2
prod 07,6
prod 08,1
prod 09,2


Dessa forma, consigo fazer uma contagem diferente para cada produto, sem precisar
fazer diversas querys, algo muito útil.

## Exercício 7

Quanto cada cliente gastou?

In [32]:
%%sql

SELECT cl.nome, round(sum(po.preço)) as gasto
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by cl.nome


 * sqlite:///TesteSQL.db
Done.


nome,gasto
Alícia Araújo,131.0
André da Cruz,108.0
Augusto Costa,79.0
Benjamin Azevedo,173.0
Brenda Porto,80.0
Bruno da Rosa,106.0
Eduarda Gonçalves,126.0
Emanuel das Neves,151.0
Enrico Gonçalves,101.0
Isis Ramos,154.0


# Order by

Muitas vezes queremos os dados em ordem específicas, por exemplo, ao visitar uma
loja online, você não espera ver produtos caros e desinteressantes sendo exibidos
para você aleatoriamente, você espera ver em uma ordem lógica, sejam os mais
baratos primeiros, os mais relevantes para o seu perfil, ou até alfabéticamente.


Tudo isso pode ser feito utilizando o comando `ORDER BY`, mas misturado com
outras operações como o `LIMIT` ele pode fazer coisas ainda melhores, como
por exemplo, trazer os 10 produtos mais caros:

In [33]:
%%sql

SELECT produto, preço
FROM produtos
ORDER BY preço DESC
LIMIT 10

 * sqlite:///TesteSQL.db
Done.


produto,preço
prod 45,9.74
prod 27,9.68
prod 24,9.57
prod 91,9.56
prod 03,9.55
prod 99,9.55
prod 18,9.46
prod 23,9.44
prod 34,9.37
prod 38,9.3


Com isso podemos ver o poder do `ORDER BY` + `LIMIT` para trazer os N primeiros
registros de uma determinada métrica

## Exercício 8

Traga os 10 clientes que mais gastaram

In [34]:
%%sql

SELECT cl.nome
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by cl.nome
order by sum(po.preço)
LIMIT 10

 * sqlite:///TesteSQL.db
Done.


nome
Augusto Costa
Brenda Porto
Sr. Nicolas Cardoso
Enrico Gonçalves
Sra. Laura Pinto
Bruno da Rosa
André da Cruz
Srta. Ana Beatriz Jesus
João Miguel Lopes
Marcos Vinicius Novaes


O order by junto do limit é extremamente importante para fazer paginação de dados,
por exemplo, novamente ao acessar o site de compras, os produtos não são todos
exibidos de uma vez para você, do contrário, a memória do seu computador iria
para o espaço, para evitar isso, tem aqueles botões na parte de baixo para ver
a próxima página, e com isso, você pode avançar e recuar.

Digamos que um cliente viu os 10 produtos mais caros e não se interessou por
nenhum e quer ver a próxima página, ele pode fazer:

In [35]:
%%sql

SELECT produto, preço
FROM produtos
ORDER BY preço DESC
LIMIT 10,10

 * sqlite:///TesteSQL.db
Done.


produto,preço
prod 20,9.01
prod 01,8.88
prod 67,8.88
prod 62,8.56
prod 63,8.42
prod 30,8.28
prod 64,8.27
prod 53,8.09
prod 04,7.93
prod 79,7.86


O primeiro 10, representa de onde começa e o segundo quantos registros vai trazer.
Você pode mexer nos valores acima e testar a vontade para ver como muda.

## Exercício 9

Traga o 11º cliente que mais gastou

In [36]:
%%sql

SELECT cl.nome
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by cl.nome
order by sum(po.preço)
LIMIT 10,01

 * sqlite:///TesteSQL.db
Done.


nome
Eduarda Gonçalves


# Funções janeladas

As funções janeladas ou funções analíticas como também são conhecidas, são funções
que se aplicam a todas a coluna trazendo métricas como qual posição essa pessoa está
entre as que mais gastaram, ou quantas pessoas no mínimo foram necessárias para 
alcançar até 25% da receita total.

In [37]:
%%sql

SELECT produto, preço, rank() over (order by preço desc) as posição
FROM produtos
order by posição
limit 10

 * sqlite:///TesteSQL.db
Done.


produto,preço,posição
prod 45,9.74,1
prod 27,9.68,2
prod 24,9.57,3
prod 91,9.56,4
prod 03,9.55,5
prod 99,9.55,5
prod 18,9.46,7
prod 23,9.44,8
prod 34,9.37,9
prod 38,9.3,10


Você pode usar as funções janeladas quando lhe for conveniente, mas não serão
obrigatórias em nenhum exercício

## Exercício 10

Qual foi o produto que menos vendeu?

In [38]:
%%sql
SELECT produto, rank() over (order by sum(po.preço) desc ) as menos_vendido
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID

 * sqlite:///TesteSQL.db
Done.


produto,menos_vendido
prod 19,1


# Operadores lógicos comuns

Alguns operadores lógicos bem famosos são o `=`, `>`, `<`, `>=`, `<=`, `!=`,
mas existem outros menos comuns, como o `LIKE` e `IN`.

O primeiro, serve para fazer pesquisas simples dentro de texto, ele irá verificar
caracter a caracter se a string informada bate com o padrão, de forma mais prática

```SQL
'abc' LIKE 'abc' -- Verdadeiro

'abcd' LIKE 'abc' -- Falso

'cbc' LIKE 'abc' -- Falso

'dabc' LIKE 'abc' -- Falso
```

Até aí é exatamente igual ao operador `=`, mas quando adicionamos os wildcards
`%` e `_` que o `LIKE` mostra o seu valor.

O wildcard `_` pode ser usado para representar qualquer caracter, por exemplo:

```SQL
'abc' LIKE 'ab_' -- Verdadeiro
'abd' LIKE 'ab_' -- Verdadeiro
'ab0' LIKE 'ab_' -- Verdadeiro
'ab ' LIKE 'ab_' -- Verdadeiro
'abcd' LIKE 'ab_' -- Falso
'cbc' LIKE 'ab_' -- Falso
'ab' LIKE 'ab_' -- Falso
'dabc' LIKE 'ab_' -- Falso
```

Já o wildcard `%` pode ser usado para representar qualquer sequência de caracteres,
por exemplo:

```SQL
'abc' LIKE 'ab%' -- Verdadeiro
'abd' LIKE 'ab%' -- Verdadeiro
'ab0' LIKE 'ab%' -- Verdadeiro
'ab ' LIKE 'ab%' -- Verdadeiro
'abcd' LIKE 'ab%' -- Verdadeiro
'cbc' LIKE 'ab%' -- Falso
'ab' LIKE 'ab%' -- Verdadeiro
'dabc' LIKE 'ab%' -- Falso
```

Usando o nosso exemplo, vamos pegar todos os clientes que tenham e-mail uol:

In [39]:
%%sql

SELECT *
FROM CLIENTES
WHERE EMAIL LIKE '%@uol.com.br'

 * sqlite:///TesteSQL.db
Done.


id,nome,email,idade
13,Sr. Renan Castro,luiz-otavio73@uol.com.br,40
17,João Miguel Lopes,rdas-neves@uol.com.br,35


Já o operador `IN` serve para verificar se o valor está dentro de um cojunto, por exemplo

```SQL
1 in (1, 2, 3) -- Verdadeiro
2 in (1, 2, 3) -- Verdadeiro
3 in (1, 2, 3) -- Verdadeiro
4 in (1, 2, 3) -- Falso
(1, 2) in ((1, 2), (3, 4)) -- Verdadeiro
(1, 3) in ((1, 2), (3, 4)) -- Falso
```

In [40]:
%%sql

SELECT *
FROM PRODUTOS
WHERE PRODUTO IN ('prod 01', 'prod 20', 'batata')

 * sqlite:///TesteSQL.db
Done.


id,produto,preço
1,prod 01,8.88
20,prod 20,9.01


# Subquerys

Subquerys são utilizadas para fazer extrações mais complexas, quando para 
alcançar um resultado, é necessário outro resultado, digamos, listar todos os
produtos comprados pelo cliente que mais gastou, primeiro precisamos encotrar o 
cliente que mais gastou e depois os produtos que ele comprou.

Podemos separar isso em duas querys:

1. Selecionar o cliente que mais gastou
2. Selecionar todos os produtos que um cliente comprou

E elas ficariam como:

In [41]:
%%sql

-- Selecionando o cliente que mais gastou

SELECT cl2.ID, cl2.nome, sum(po2.preço) AS GASTO
FROM clientes cl2
JOIN compras AS co2
ON co2.ID_CLIENTE = cl2.ID
JOIN produtos as po2
ON co2.ID_PRODUTO = po2.ID
group by cl2.nome, cl2.id
ORDER BY gasto DESC
LIMIT 1

 * sqlite:///TesteSQL.db
Done.


id,nome,GASTO
11,Benjamin Azevedo,173.45000000000002


In [42]:
%%sql

-- Selecionar todos os produtos que umm cliente comprou

SELECT DISTINCT po.produto
FROM produtos as po
JOIN compras AS co
ON co.ID_PRODUTO = po.ID
WHERE co.ID_CLIENTE IN (5)

 * sqlite:///TesteSQL.db
Done.


produto
prod 93
prod 52
prod 23
prod 43
prod 46
prod 77
prod 29
prod 45
prod 95
prod 53


In [43]:
%%sql

-- Juntando as duas:
    
SELECT DISTINCT po.produto
FROM produtos as po
JOIN compras AS co
ON co.ID_PRODUTO = po.ID
WHERE co.ID_CLIENTE IN (
    SELECT cl2.ID
    FROM clientes cl2
    JOIN compras AS co2
    ON co2.ID_CLIENTE = cl2.ID
    JOIN produtos as po2
    ON co2.ID_PRODUTO = po2.ID
    group by cl2.nome, cl2.id
    ORDER BY sum(po2.preço) DESC
    LIMIT 1
)

 * sqlite:///TesteSQL.db
Done.


produto
prod 90
prod 37
prod 88
prod 05
prod 30
prod 62
prod 48
prod 04
prod 10
prod 18


Se eu quisesse exibir o nome do cliente, eu poderia fazer mais um join pegando
o nome dele na tabela cliente, mas nesse caso, eu estou pegando um dado que eu
consigo facilmente na subquery, existem formas de trazer dados lá de dentro,
como por  exemplo:

In [44]:
%%sql

    
SELECT tb.nome, po.produto
FROM produtos as po
JOIN compras AS co
ON co.ID_PRODUTO = po.ID
join (
    SELECT cl2.ID, cl2.nome
    FROM clientes cl2
    JOIN compras AS co2
    ON co2.ID_CLIENTE = cl2.ID
    JOIN produtos as po2
    ON co2.ID_PRODUTO = po2.ID
    group by cl2.nome, cl2.id
    ORDER BY sum(po2.preço) DESC
    LIMIT 1
) tb on co.ID_CLIENTE = tb.ID
group by tb.nome, po.produto

 * sqlite:///TesteSQL.db
Done.


nome,produto
Benjamin Azevedo,prod 02
Benjamin Azevedo,prod 04
Benjamin Azevedo,prod 05
Benjamin Azevedo,prod 10
Benjamin Azevedo,prod 12
Benjamin Azevedo,prod 13
Benjamin Azevedo,prod 14
Benjamin Azevedo,prod 18
Benjamin Azevedo,prod 20
Benjamin Azevedo,prod 30


Não existe limite de quantidade de subquerys que você pode fazer, sejam uma duas
ou várias, fazendo uma verdadeira boneca russa, mas use com atenção, verifique
sempre se realmente é necessário.

Se por exemplo, eu quisesse saber quantos clientes compraram os mesmos
produtos que a pessoa que mais gastou comprou


1. Selecionar o cliente que mais gastou
2. Selecionar todos os produtos que um cliente comprou
3. Contar quantos clientes compraram um produto

Como a query 1 e 2 já são antigas, vou fazer a 3

In [45]:
%%sql

-- Contar quantos clientes compraram um produto

SELECT po.produto, count(DISTINCT co.ID_CLIENTE)
FROM compras co
JOIN produtos po on po.ID = co.ID_PRODUTO
WHERE co.ID_PRODUTO IN (5)
GROUP by po.produto

 * sqlite:///TesteSQL.db
Done.


produto,count(DISTINCT co.ID_CLIENTE)
prod 05,6


In [46]:
%%sql

SELECT po.produto, count(DISTINCT co.ID_CLIENTE)
FROM compras co
JOIN produtos po on po.ID = co.ID_PRODUTO
WHERE co.ID_PRODUTO IN (
    SELECT DISTINCT po2.id
    FROM produtos as po2
    JOIN compras AS co2
    ON co2.ID_PRODUTO = po2.ID
    WHERE co2.ID_CLIENTE IN (
        SELECT cl3.ID
        FROM clientes cl3
        JOIN compras AS co3
        ON co3.ID_CLIENTE = cl3.ID
        JOIN produtos as po3
        ON co3.ID_PRODUTO = po3.ID
        group by cl3.nome, cl3.id
        ORDER BY sum(po3.preço) DESC
        LIMIT 1
    )
)
GROUP by po.produto

 * sqlite:///TesteSQL.db
Done.


produto,count(DISTINCT co.ID_CLIENTE)
prod 02,2
prod 04,7
prod 05,6
prod 10,5
prod 12,5
prod 13,5
prod 14,6
prod 18,7
prod 20,3
prod 30,7


Mas novamente podemos simplificar usando join ao invés de where, lembre-se,
quanto menos vezes você repetir o nome de uma tabela, melhor

In [47]:
%%sql

SELECT po.produto, count(DISTINCT co.ID_CLIENTE)
FROM compras co
JOIN produtos as po on co.ID_PRODUTO = po.ID
JOIN (
    SELECT cl3.ID
    FROM clientes cl3
    JOIN compras AS co3
    ON co3.ID_CLIENTE = cl3.ID
    JOIN produtos as po3
    ON co3.ID_PRODUTO = po3.ID
    group by cl3.nome, cl3.id
    ORDER BY sum(po3.preço) DESC
    LIMIT 1
) q1
JOIN(
    SELECT co2.id_PRODUTO, co2.ID_CLIENTE
    FROM compras AS co2
    GROUP BY co2.id_PRODUTO, co2.ID_CLIENTE
) q2 ON co.ID_PRODUTO = q2.ID_PRODUTO AND q2.ID_CLIENTE = q1.ID
GROUP by po.produto

 * sqlite:///TesteSQL.db
Done.


produto,count(DISTINCT co.ID_CLIENTE)
prod 02,2
prod 04,7
prod 05,6
prod 10,5
prod 12,5
prod 13,5
prod 14,6
prod 18,7
prod 20,3
prod 30,7


# With

Muitas vezes fica confuso ficar colocando várias subquerys no meio da query 
pricipal, ou você quer repetir ela, não quer ter que trocar alias e etc.

Para resolver esses problemas, ou até outros, consulte a 
[documentação](https://www.sqlite.org/lang_with.html) para ver
tudo o que ele é capaz, existe o `WITH`.

In [48]:
%%sql

WITH q1 AS (
    SELECT cl.ID
    FROM clientes cl
    JOIN compras AS co
    ON co.ID_CLIENTE = cl.ID
    JOIN produtos as po
    ON co.ID_PRODUTO = po.ID
    group by cl.nome, cl.id
    ORDER BY sum(po.preço) DESC
    LIMIT 1
),
q2 AS (
    SELECT co.id_PRODUTO, co.ID_CLIENTE
    FROM compras AS co
    GROUP BY co.id_PRODUTO, co.ID_CLIENTE
)
SELECT po.produto, count(DISTINCT co.ID_CLIENTE)
FROM compras co
JOIN produtos as po on co.ID_PRODUTO = po.ID
JOIN q1
JOIN q2 ON co.ID_PRODUTO = q2.ID_PRODUTO AND q2.ID_CLIENTE = q1.ID
GROUP by po.produto

 * sqlite:///TesteSQL.db
Done.


produto,count(DISTINCT co.ID_CLIENTE)
prod 02,2
prod 04,7
prod 05,6
prod 10,5
prod 12,5
prod 13,5
prod 14,6
prod 18,7
prod 20,3
prod 30,7


## Exercício 11

Traga todos os clientes que compraram o produto que menos vendeu

In [50]:
%%sql


    
SELECT DISTINCT cl.nome, po.produto      -- todos os clientes que compraram
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID 
WHERE co.ID_PRODUTO IN (             -- colocar o mesmo no select
SELECT  co2.ID_PRODUTO              -- produto que menos vendeu
    FROM clientes cl2
    JOIN compras AS co2
    ON co2.ID_CLIENTE = cl2.ID
    JOIN produtos as po2
    ON co2.ID_PRODUTO = po2.ID
    group by po2.produto
    order by  count(*)
    limit 1
)

 * sqlite:///TesteSQL.db
Done.


nome,produto
Emanuel das Neves,prod 08


## Exercício 12

Traga quanto gastou cada cliente que comprou o produto que menos vendeu

In [52]:
%%sql

WITH quantidade_de_produto_mv as (         
	SELECT count(*) as qtd
	FROM compras AS co
	group by co.id_produto
	order by qtd
	limit 1	
),
produto_que_menos_vendeu AS (                                 -- produto que menos vendeu
	SELECT co.ID_PRODUTO, count(*) as qtd
	FROM compras AS co
	group by co.id_produto
	having qtd = (SELECT qtd from quantidade_de_produto_mv)
),
oq_cd_1_comprou as (                               -- o que cada cliente comprou
	SELECT
		cl.nome, 
		co.id_cliente,
		po.produto,
		co.id_produto,
		po.preço 
	FROM clientes cl
	JOIN compras AS co ON co.ID_CLIENTE = cl.ID
	JOIN produtos as po ON co.ID_PRODUTO = po.ID
), quanto_cada_um_gastou as (                          -- quanto gastou cada cliente
   SELECT 
    id_cliente,
    nome,
 	sum(preço) as gasto
   FROM oq_cd_1_comprou
   group by id_cliente, nome
 )
 , quem_comprou_menos_vendeu as (               -- quem comprou o que menos vendeu
	select 
 	DISTINCT nome, id_cliente 
 	FROM oq_cd_1_comprou as occ
 	JOIN produto_que_menos_vendeu as pmv ON occ.id_produto = pmv.id_produto
 )
  SELECT                                  -- quanto gastou cada cliente que comprou o produto que menos vendeu
  qcmv.nome, qcmv.id_cliente, gasto
  FROM quanto_cada_um_gastou as qcug
  JOIN quem_comprou_menos_vendeu as qcmv ON   qcug.id_cliente = qcmv.id_cliente

 * sqlite:///TesteSQL.db
Done.


nome,id_cliente,gasto
Emanuel das Neves,3,150.98999999999998
Sr. Nicolas Cardoso,5,99.44999999999999
Alícia Araújo,6,131.49999999999997
Brenda Porto,9,80.25
Isis Ramos,19,153.81


## Exercício 13

Quanto em média os clientes gastaram?

In [53]:
%%sql

SELECT sum(po.preço)/count(distinct co.id_cliente) as média
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID

 * sqlite:///TesteSQL.db
Done.


média
123.708


## Exercício 14

Quanto em média gastou cada cliente?

In [54]:
%%sql

SELECT cl.nome, sum(po.preço)/count(*) as média
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by cl.nome


 * sqlite:///TesteSQL.db
Done.


nome,média
Alícia Araújo,4.870370370370369
André da Cruz,4.675652173913043
Augusto Costa,4.392222222222222
Benjamin Azevedo,5.781666666666667
Brenda Porto,5.35
Bruno da Rosa,5.585789473684212
Eduarda Gonçalves,5.2437499999999995
Emanuel das Neves,4.870645161290322
Enrico Gonçalves,4.807619047619048
Isis Ramos,4.8065625


## Exercício 15

Quantos clientes tem cada faixa etária (Agrupe por faixa de 10 anos)

i.e.: \[18 -> 19\], \[20 -> 29\], \[30 -> 39\], ...

In [55]:
%%sql

SELECT cast(idade/10 AS integer)*10 as faixa_etaria, count(*)
FROM clientes cl
group by faixa_etaria

 * sqlite:///TesteSQL.db
Done.


faixa_etaria,count(*)
20,3
30,9
40,6
50,2


## Exercício 16

Qual é o gasto médio para cada faixa etária?

In [56]:
%%sql

SELECT cast(idade/10 AS integer)*10 as faixa_etaria, sum(po.preço)/count(distinct co.id_cliente) as gasto_médio
FROM clientes cl
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by faixa_etaria

 * sqlite:///TesteSQL.db
Done.


faixa_etaria,gasto_médio
20,138.8433333333333
30,113.59333333333328
40,126.37333333333332
50,138.52499999999998


## Exercício 17

Quais foram os 5 produtos que mais venderam na faixa etária que teve maior
gasto médio?

In [59]:
%%sql

SELECT po.produto, count(*)            -- 5 produtos que mais venderam
FROM clientes cl
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by po.produto
order by  count(*) DESC
limit 5


SELECT cast(idade/10 AS integer)*10 as faixa_etaria, sum(po.preço)/count(distinct co.id_cliente) as gasto_médio   -- MAIOR GASTO MÉDIO
FROM clientes cl
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by faixa_etaria
order by gasto_médio DESC
limit 1




 * sqlite:///TesteSQL.db
(sqlite3.OperationalError) near "SELECT": syntax error
[SQL: SELECT po.produto, count(*)            -- 5 produtos que mais venderam
FROM clientes cl
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by po.produto
order by  count(*) DESC
limit 5


SELECT cast(idade/10 AS integer)*10 as faixa_etaria, sum(po.preço)/count(distinct co.id_cliente) as gasto_médio   -- MAIOR GASTO MÉDIO
FROM clientes cl
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by faixa_etaria
order by gasto_médio DESC
limit 1]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


## Exercício 18

Quais foram os 5 produtos que mais venderam na faixa etária que teve maior
gasto absoluto?

In [60]:
%%sql

SELECT po.produto, count(*)            -- 5 produtos que mais venderam
FROM clientes cl
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by po.produto
order by  count(*) DESC
limit 5




SELECT cast(idade/10 AS integer)*10 as faixa_etaria, sum(po.preço) as gasto_absoluto   -- MAIOR GASTO ABSOLUTO
FROM clientes cl
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by faixa_etaria
order by gasto_absoluto DESC
limit 1

 * sqlite:///TesteSQL.db
(sqlite3.OperationalError) near "SELECT": syntax error
[SQL: SELECT po.produto, count(*)            -- 5 produtos que mais venderam
FROM clientes cl
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by po.produto
order by  count(*) DESC
limit 5




SELECT cast(idade/10 AS integer)*10 as faixa_etaria, sum(po.preço) as gasto_absoluto   -- MAIOR GASTO ABSOLUTO
FROM clientes cl
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by faixa_etaria
order by gasto_absoluto DESC
limit 1]
(Background on this error at: https://sqlalche.me/e/14/e3q8)
